In [1]:
import pandas as pd
import numpy as np
import requests
import re as regex
import random
import math
import string

preprocessing

In [2]:
tweets_input = requests.get('https://raw.githubusercontent.com/avi20025/Machine-Learning/main/usnewshealth.txt')
tweets_input = tweets_input.text
tweets_input = tweets_input.split('\n')

print(len(tweets_input))

1401


In [3]:
tweet_start_index = 50

In [4]:
def preprocess_tweets(tweets):

    processed_data = []

    for tweets in tweets_input:

        # Remove tweet id and timestamp
        tweet = tweets[tweet_start_index:]

        # Remove words starting with '@'
        tweet = regex.sub('@[^ \t]+', '', tweet)

        # Remove hashtag symbols
        tweet = regex.sub('#', '', tweet)

        # Remove URLs
        tweet = regex.sub('(http|https)://[^\s]*', '', tweet)

        # Convert to lowercase
        tweet = tweet.lower()

        tweet = tweet.split()

        processed_data.append(tweet)

    #for tweet in processed_data[:30]:
        #print(tweet)

    return processed_data

In [5]:
def jaccard_distance(tweet1, tweet2):
    # Finding count of intersection between two tweets
    intersection = len(list(set(tweet1)&set(tweet2)))

    # Finding count of union between two tweets
    union = len(tweet1) + len(tweet2) - intersection

    # Returning jaccard distance
    jdistance = (1 - (intersection / union))

    return jdistance

In [6]:
def assign_to_clusters(data, centroids):

    cluster = dict()

    tweet = 0
    # for all tweets, search for centroids and assign it to the closest centroid
    while tweet < len(data):
        minimum_distance = math.inf
        tweets = data[tweet]
        C_index = -1

        i = 0
        # find the closest centroid
        while i < len(centroids):
            centroid = centroids[i]
            distance = jaccard_distance(centroid, data)

            if centroid == tweets:
                C_index = i
                minimum_distance = 0
                break

            if distance < minimum_distance:
                C_index = i
                minimum_distance = distance

            i += 1

        # randomise centroid assignment to a tweet if there is nothing in common
        if minimum_distance == 1:
            C_index = random.randint(0, len(centroids) - 1)


        clustertweet = cluster.setdefault(C_index, [])
        clustertweet.append([tweets])

        t_index = len(clustertweet) - 1
        clustertweet[t_index].append(minimum_distance)

        tweet += 1

    return cluster

In [7]:
def update_centroids(clusters):

    centroids = []

    i = 0
    # iterate over each cluster to check for a tweet with closest distance sum with every other tweets present in the same cluster
    while i < len(clusters):
      min_dists = math.inf
      centroid_index = -1

      min_dists_arr = []

      tweet1 = 0
      while tweet1 < len(clusters[i]):
        min_dists_arr.append([])
        dist_sum = 0

        tweet2 = 0
        # calculates distance sum for every pair of tweets in the same cluster
        while tweet2 < len(clusters[i]):
          if tweet1 != tweet2:
            if tweet2 > tweet1:
              distance = jaccard_distance(clusters[i][tweet1][0], clusters[i][tweet2][0])
            else:
              distance = min_dists_arr[tweet2][tweet1]


            min_dists_arr[tweet1].append(distance)
            dist_sum += distance
          else:
            min_dists_arr[tweet1].append(0)

          tweet2 += 1

        # identifies and selects tweet with minimum distance sum as centroid for current cluster
        if dist_sum < min_dists:
          min_dists = dist_sum
          centroid_index = tweet1

        tweet1 += 1

      # puts identified tweet into centroid list
      centroids.append(clusters[i][centroid_index][0])

      i += 1

    return centroids

In [8]:
def kmeans(data, k , max_iter=35):
    # Randomize the seed
    random.seed()

    oldCentroids = []
    iteration = 0

    # Initializing by randomly selecting k data as starting centroids
    centroids = random.sample(data, k)

    # iterate until max_iteration or converged
    while (iteration < max_iter) and (not check_Converge(oldCentroids, centroids)) :
        print(f"Iteration number {iteration}")

        # assign data to clusters
        clusters = assign_to_clusters(data, centroids)

        # compare with previous centroids to check if converging
        oldCentroids = centroids

        # updating centroid with clusters
        centroids = update_centroids(clusters)
        iteration += 1

    if iteration == max_iter:
        print("it reached maximum number of iterations, it did not converge..")
    else:
        print("it converged..")

    sse = calculate_sse(clusters)

    return clusters, sse

In [9]:
def calculate_sse(clusters):
    sse = 0

    # iterate over clusters and compute sum of squared error of distances of tweets from its centroid
    for i in range(len(clusters)):
      for j in range(len(clusters[i])):
        sse += (clusters[i][j][1] * clusters[i][j][1])

    return sse

In [10]:
def check_Converge(old_centroid, new_centroids):
    # check if the lengths are same and return false if so
    if len(old_centroid) != len(new_centroids):
        return False

    # check if each pair of centroids are equal
    for old, current in zip(old_centroid, new_centroids):
        if " ".join(current) != " ".join(old):
            return False

    return True

In [12]:
#preprocess the tweets
preprocess_tweets(tweets_input)
tweets = tweets_input

# initial K value for Kmeans, change if needed
k=5

print("K means for k = {}".format(k))
clusters, sse = kmeans(tweets, k)
for c in range(len(clusters)):
            print(str(c+1) + ": ", str(len(clusters[c])) + " tweets")
print("SSE == {} ".format(sse))


K means for k = 5
Iteration number 0
Iteration number 1
Iteration number 2
it converged..
1:  282 tweets
2:  293 tweets
3:  266 tweets
4:  261 tweets
5:  299 tweets
SSE == 1396.0 
